# CDPH variant totals by type

Downloads [variant totals](https://covid19.ca.gov/variants/) published by the California Department of Public Health.

## Import

In [2]:
import os
import glob
import pytz
import requests
import pandas as pd
from datetime import datetime

In [3]:
tz = pytz.timezone("America/Los_Angeles")

In [4]:
today = datetime.now(tz).date()

## Scrape

In [5]:
url = "https://data.covid19.ca.gov/data/variants/california.json"

In [6]:
response = requests.get(url)

In [7]:
data = response.json()

In [8]:
pub_date = data["meta"]["PUBLISHED_DATE"]

In [9]:
report_date = data["meta"]["REPORT_DATE"]

## Parse

In [10]:
df_list = []
for key, value in data["data"]["time_series"].items():
    df = pd.DataFrame(value["VALUES"])
    df.rename(columns={"VALUE": key}, inplace=True)
    df_list.append(df.set_index("DATE"))

In [11]:
df = pd.concat(df_list, axis=1)

In [12]:
df["published_date"] = pub_date

In [13]:
df["report_date"] = report_date

In [14]:
df["scrape_date"] = today

## Export

In [15]:
data_dir = os.path.join(os.path.abspath(""), "data")

In [16]:
df.to_csv(os.path.join(data_dir, "timeseries.csv"))